# 💰 Counting Other People`s Money with PowerShell

You thought PowerShell was just for managing servers? Think again! Today, we`re going to engage in a noble pursuit: **counting other people’s money**. And not just anywhere — we’re diving into the official filings of the U.S. Securities and Exchange Commission (SEC). All from the comfort of the console, with a splash of [Vega](https://vega.github.io) and a hint of analytical mischief.

Our target of curiosity: Form 4, where corporate big shots report their stock transactions:
- Sold something? Gotta report it.
- Gifted shares to the spouse? Still report it.
- Got a bonus in shares? Even if it’s “just a thank you” — report it!

Why do we care? Well… we’re just curious who dumped their shares right before the price tanked 😉

To do this, we`ll need two trusty PowerShell functions:
- **Get-RecentSecForm4XmlUrls** — our investigator, crawling through the SEC archives to extract URLs to XML filings.
- **Convert-Form4XmlToRecord** — parses the XML and turns it into a proper PowerShell object. Because reading raw XML? Painful. Let the script suffer.


In [1]:
<#
.SYNOPSIS
Retrieves a list of XML URLs for recent Form 4 insider filings from the SEC for a specified CIK.

.DESCRIPTION
This function queries the SEC EDGAR submissions API for a given company identified by its CIK (Central Index Key),
and returns a list of links to XML versions of Form 4 insider trading reports filed within the past N days.

.PARAMETER CIK
The Central Index Key (CIK) of the company. Defaults to Microsoft Corporation (0000789019).

.PARAMETER DaysBack
Number of days in the past to include filings. Defaults to 100 days.

.OUTPUTS
[PSCustomObject] with the following fields:
- FilingDate
- ReportDate
- XmlUrl

.EXAMPLE
Get-RecentSecForm4XmlUrls -CIK "0000320193" -DaysBack 30
Retrieves recent Form 4 XML links for Apple Inc. over the past 30 days.

.EXAMPLE
Get-RecentSecForm4XmlUrls
Returns recent Form 4 filings for Microsoft Corporation from the past 100 days.

.NOTES
A custom User-Agent header is required to access the SEC data endpoints.
#>

function Get-RecentSecForm4XmlUrls {
    param (
        [string]$CIK = "0000789019",
        [int]$DaysBack = 100
    )

    $headers = @{
        "User-Agent" = "PowerShellScript/1.0 (eosfor@gmail.com)"
        "Accept-Encoding" = "gzip, deflate"
    }

    $url = "https://data.sec.gov/submissions/CIK$CIK.json"
    $data = Invoke-RestMethod -Uri $url -Headers $headers

    $cikTrimmed = $CIK.TrimStart("0")
    $cutoffDate = (Get-Date).AddDays(-$DaysBack)

    $results = @()

    for ($i = 0; $i -lt $data.filings.recent.form.Length; $i++) {
        $formType = $data.filings.recent.form[$i]
        if ($formType -ne "4") { continue }

        $filingDate = Get-Date $data.filings.recent.filingDate[$i]
        if ($filingDate -lt $cutoffDate) { continue }

        $accessionNumber = $data.filings.recent.accessionNumber[$i]
        $primaryDoc = $data.filings.recent.primaryDocument[$i]
        $reportDate = $data.filings.recent.reportDate[$i]

        $folder = $accessionNumber -replace "-", ""
        $xmlFileName = [System.IO.Path]::GetFileNameWithoutExtension($primaryDoc) + ".xml"
        $xmlUrl = "https://www.sec.gov/Archives/edgar/data/$cikTrimmed/$folder/$xmlFileName"

        $results += [PSCustomObject]@{
            FilingDate = $filingDate.ToString("yyyy-MM-dd")
            ReportDate = $reportDate
            XmlUrl     = $xmlUrl
        }
    }

    return $results
}

In [2]:
<#
.SYNOPSIS
Converts a Form 4 XML document into a structured PowerShell object representing insider transactions.

.DESCRIPTION
This function takes an object with an XmlUrl (typically output from Get-RecentSecForm4XmlUrls), downloads the Form 4 XML,
and extracts detailed information about the issuer, insider, role, transaction type, number of shares, price,
ownership nature, and any associated footnotes.

.PARAMETER InputObject
An object containing XmlUrl, FilingDate, and ReportDate fields. Usually piped from Get-RecentSecForm4XmlUrls.

.OUTPUTS
[PSCustomObject] with the following fields:
- FilingDate
- ReportDate
- Issuer
- InsiderName
- InsiderRole
- SecurityTitle
- TransactionDate
- TransactionCode
- SharesTransacted
- PricePerShare
- SharesOwnedAfterTxn
- OwnershipType
- IndirectOwnershipNature
- Footnote
- XmlUrl

.EXAMPLE
Get-RecentSecForm4XmlUrls -CIK "0000789019" | Convert-Form4XmlToRecord
Returns parsed insider transactions for Microsoft Corporation.

.NOTES
Only non-derivative transactions are processed. If the XML cannot be downloaded, a warning is displayed.
#>

function Convert-Form4XmlToRecord {
    [CmdletBinding()]
    param (
        [Parameter(ValueFromPipeline = $true)]
        [pscustomobject]$InputObject
    )

    process {
        $headers = @{
            "User-Agent" = "PowerShellScript/1.0 (eosfor@gmail.com)"
        }

        try {
            [xml]$doc = Invoke-WebRequest -Uri $InputObject.XmlUrl -Headers $headers -UseBasicParsing
        }
        catch {
            Write-Warning "Download failed: $($InputObject.XmlUrl)"
            return
        }

        $issuer = $doc.ownershipDocument.issuer.issuerName
        $owner = $doc.ownershipDocument.reportingOwner.reportingOwnerId.rptOwnerName
        $ownerRelationship = $doc.ownershipDocument.reportingOwner.reportingOwnerRelationship
        $relationship = ($ownerRelationship | Get-Member -MemberType Properties | Where-Object { $ownerRelationship.$($_.Name) -eq "1" }).Name

        # Собираем footnotes в хештаблицу
        $footnotes = @{}

        if ($doc.ownershipDocument.footnotes -and $doc.ownershipDocument.footnotes.footnote) {
            $rawFootnotes = $doc.ownershipDocument.footnotes.footnote

            # Убедимся, что это массив
            if ($rawFootnotes -is [System.Array]) {
                foreach ($f in $rawFootnotes) {
                    $footnotes[$f.id] = $f.'#text' ?? $f.InnerText
                }
            }
            else {
                $footnotes[$rawFootnotes.id] = $rawFootnotes.'#text' ?? $rawFootnotes.InnerText
            }
        }

        $transactions = $doc.ownershipDocument.nonDerivativeTable.nonDerivativeTransaction
        foreach ($txn in $transactions) {
            # если есть одна или несколько сносок — собираем их все
            $note = $null
            if ($txn.footnoteId) {
                $ids = if ($txn.footnoteId -is [System.Array]) {
                    $txn.footnoteId | ForEach-Object { $_.id }
                } else {
                    @($txn.footnoteId.id)
                }

                $note = ($ids | ForEach-Object { $footnotes[$_] }) -join "; "
            }

            [PSCustomObject]@{
                FilingDate              = $InputObject.FilingDate
                ReportDate              = $InputObject.ReportDate
                Issuer                  = $issuer
                InsiderName             = $owner
                InsiderRole             = $relationship
                SecurityTitle           = $txn.securityTitle.value
                TransactionDate         = $txn.transactionDate.value
                TransactionCode         = $txn.transactionCoding.transactionCode
                SharesTransacted        = $txn.transactionAmounts.transactionShares.value
                PricePerShare           = $txn.transactionAmounts.transactionPricePerShare.value
                SharesOwnedAfterTxn     = $txn.postTransactionAmounts.sharesOwnedFollowingTransaction.value
                OwnershipType           = $txn.ownershipNature.directOrIndirectOwnership.value
                IndirectOwnershipNature = $txn.ownershipNature.natureOfOwnership.value
                Footnote                = $note
                XmlUrl                  = $InputObject.XmlUrl
            }
        }
    }
}

📥 Let's fire up our surveillance script and stash the data in a variable named $allData. Think of it as “doing a background check,” but legally. You can put your CIK here

In [3]:
$allData = 
Get-RecentSecForm4XmlUrls -CIK "0000789019" -DaysBack 107 |
    Convert-Form4XmlToRecord

🧹 Next step — let's clean house. We only care about transactions where money actually moved. If the number of shares is 0 — skip it. We`re here for the real million-dollar moves (or at least a few solid trades).

In [4]:
$data = $allData |
    Select-Object TransactionDate, SharesTransacted, TransactionCode |
    Where-Object { $_.TransactionCode -in @("S", "P", "F", "A", "M", "G") -and $_.SharesTransacted -gt 0 }

$data = $data | ForEach-Object {
    $action = switch ($_.TransactionCode) {
        "S" { "Sell"; break }
        "F" { "Sell"; break }
        "G" { "Sell"; break }
        "A" { "Buy"; break }
        "P" { "Buy"; break }
        "M" { "Buy"; break }
        default { "Other" }
    }

    $_ | Add-Member -NotePropertyName Action -NotePropertyValue $action -Force -PassThru
}

📊 Time to aggregate who bought or sold how much. We’ll group by insider name and transaction type. If the price is known — we’ll calculate totals. If not — we label it as “unknown,” like old-school accounting.

In [5]:
$data2 = $allData |
    # Filter only transactions with non-zero number of shares
    Where-Object { $_.SharesTransacted -gt 0 } |

    # Group by InsiderName and TransactionCode (e.g., "John Smith|S")
    Group-Object { "$($_.InsiderName)|$($_.TransactionCode)" } |

    ForEach-Object {
        $parts = $_.Name -split '\|'     # Split group name into [InsiderName, TransactionCode]
        $group = $_.Group                # Access the actual group of transactions

        # Filter only those deals with valid numeric and positive price per share
        $validDeals = $group | Where-Object {
            [double]::TryParse($_.PricePerShare, [ref]$null) -and [double]$_.PricePerShare -gt 0
        }

        # Sum all shares transacted in the group
        $sharesSum = ($group | Measure-Object -Property SharesTransacted -Sum).Sum

        # Calculate total value by summing (Shares × Price) across valid deals
        $totalValue = ($validDeals | ForEach-Object {
            [double]$_.SharesTransacted * [double]$_.PricePerShare
        }) | Measure-Object -Sum | Select-Object -ExpandProperty Sum

        # Display value only if valid (not null or NaN)
        $valueDisplay = if ($totalValue -and $totalValue -gt 0 -and -not [double]::IsNaN($totalValue)) {
            [math]::Round($totalValue, 2)
        } else {
            "unknown"
        }

        # Return summary object for each (InsiderName, TransactionCode) group
        [PSCustomObject]@{
            Insider            = $parts[0]
            TransactionCode    = $parts[1]
            Count              = $_.Count
            TotalShares        = [math]::Round($sharesSum, 2)
            TotalValue         = if ($valueDisplay -is [string]) { $null } else { $valueDisplay }
            TotalValueDisplay  = "$valueDisplay"
        }
    }

📋 This part just breaks down each transaction into a clean row: date, who, what, how much. If the price is listed — great. If not — well, you guessed it: “unknown.” The point is, nobody escapes the ledger 🕵️‍♂️

In [6]:
$data3 = $allData |
    # Filter transactions with shares > 0 and valid date format (YYYY-MM-DD)
    Where-Object {
        $_.SharesTransacted -gt 0 -and
        $_.TransactionDate -match '^\d{4}-\d{2}-\d{2}$'
    } |

    # Process each valid transaction
    ForEach-Object {
        $value = $null              # Holds calculated total value (shares * price)
        $display = "unknown"     # Default display if value is unknown

        # Calculate total value if price is valid and > 0
        if ([double]::TryParse($_.PricePerShare, [ref]$null) -and [double]$_.PricePerShare -gt 0) {
            $value = [math]::Round([double]$_.SharesTransacted * [double]$_.PricePerShare, 2)
            $display = "$value"     # Use calculated value for display
        }

        # Return a simplified transaction record
        [PSCustomObject]@{
            Insider            = $_.InsiderName
            TransactionDate    = $_.TransactionDate
            TransactionCode    = $_.TransactionCode
            SharesTransacted   = [int]$_.SharesTransacted
            TotalValue         = $value
            TotalValueDisplay  = $display
        }
    }

💾 Final touch — export our precious data to CSV, so we can plot fancy charts and show off that PowerShell isn’t just for Get-Process.

In [7]:
$allData | Export-Csv allData.csv -NoTypeInformation
$data | Export-Csv -Path "form4-trades.csv" -NoTypeInformation
$data2 | Export-Csv -Path "insider-heatmap.csv" -NoTypeInformation
$data3 | Export-Csv -NoTypeInformation -Path "insider-scatter.csv"

🔧 Almost forgot! To make all this work smoothly, we had to contribute a little something to dotnet/interactive. Why? Because the CustomMimeType parameter in Out-Display was… well, kind of there but not really working. Now it works — JSON specs right from the notebook cell, beautiful charts and all. Feel free to thank the author of [PR #3671](https://github.com/dotnet/interactive/pull/3671), and that's, actually, me 😉

📈 Scatter Plot — our first visual interrogation:
	•	X — transaction date
	•	Y — number of shares
	•	Color — green (buy) or red (sell)
	•	Tooltip — who, when, how much, and the SEC code letter

A quick way to spot who knew what and sold just in time 💸

And this is how it looks. Pretty, informative, and just a little incriminating.

In [8]:
@"
{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "Insider Trading Scatter Plot",
  "data": {
    "url": "form4-trades.csv",
    "format": {"type": "csv"}
  },
  "mark": "point",
  "encoding": {
    "x": {
      "field": "TransactionDate",
      "type": "temporal",
      "title": "Transaction Date"
    },
    "y": {
      "field": "SharesTransacted",
      "type": "quantitative",
      "title": "Shares Transacted"
    },
    "color": {
      "field": "Action",
      "type": "nominal",
      "scale": {
        "domain": ["Buy", "Sell"],
        "range": ["green", "red"]
      },
      "title": "Transaction Type"
    },
    "tooltip": [
      {"field": "TransactionDate", "type": "temporal", "title": "Date"},
      {"field": "SharesTransacted", "type": "quantitative", "title": "Shares"},
      {"field": "TransactionCode", "type": "nominal", "title": "Code"}
    ]
  }
}
"@ | Out-Display -MimeType "application/vnd.vegalite.v5+json"

### 🔍 `TransactionCode` meaning

| Code | What it means       | How to interpret it                                                   |
|------|----------------------|------------------------------------------------------------------------|
| A    | Award                | Shares granted, usually a bonus. Like a gift card, but in stock.      |
| S    | Sale                 | Sold shares. Sometimes en masse. Often... right before a price drop.  |
| F    | Tax                  | Shares withheld to pay taxes. At least they didn’t keep those.        |
| M    | Option Exercise      | Exercised an option. Buy low, sell high — the corporate dream.        |
| G    | Gift                 | Given away. To family. Or a trust. Or a charity. No judgment here.    |
| P    | Purchase             | Bought shares. With their own money. Respect.                         |
| I    | Discretionary        | Auto-trade via plan. Legit? Depends who you ask.                      |
| C    | Conversion           | Transformed derivatives into common shares. Totally by the book.      |

🔥 Heatmap — follow the heat to find the insiders:
- X — transaction type
- Y — insider
- Color — green if we know the amount, gray if unknown
- Tooltip — how many trades, shares, and how much total value

Anyone can make mistakes — but heatmaps? They never lie. 💼

In [9]:
@"
{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "Heatmap of Insider Transaction Totals",
  "data": {
    "url": "insider-heatmap.csv",
    "format": {"type": "csv"}
  },
  "mark": "rect",
  "encoding": {
    "x": {
      "field": "TransactionCode",
      "type": "nominal",
      "title": "Transaction Type"
    },
    "y": {
      "field": "Insider",
      "type": "nominal",
      "title": "Insider",
      "sort": "-x"
    },
    "color": {
      "condition": {
        "test": "isValid(datum.TotalValue) && datum.TotalValue != ''",
        "field": "TotalValue",
        "type": "quantitative",
        "scale": { "scheme": "greens" }
      },
      "value": "#eeeeee"
    },
    "tooltip": [
      { "field": "Insider", "type": "nominal", "title": "Insider" },
      { "field": "TransactionCode", "type": "nominal", "title": "Transaction Code" },
      { "field": "Count", "type": "quantitative", "title": "Number of Trades" },
      { "field": "TotalShares", "type": "quantitative", "title": "Total Shares" },
      { "field": "TotalValueDisplay", "type": "nominal", "title": "Total Value ($)" }
    ]
  },
  "config": {
    "axis": {
      "labelFontSize": 10,
      "titleFontSize": 12
    },
    "view": {
      "stroke": "transparent"
    }
  }
}
"@ | Out-Display -MimeType "application/vnd.vegalite.v5+json"

🔵 Bubble Chart — where every bubble is a trade, and size shows how big it was. The bigger the bubble — the juicier the deal:
- X — date
- Y — who
- Size — number of shares
- Color — transaction type
- Tooltip — all the dirty details

Makes it pretty obvious who burst the greed bubble first 😄

In [10]:
@"
{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "Bubble Chart: Insider vs Date vs Transaction Type",
  "width": 800,
  "data": {
    "url": "insider-scatter.csv",
    "format": { "type": "csv" }
  },
  "transform": [
    {
      "lookup": "TransactionCode",
      "from": {
        "data": {
          "values": [
            { "code": "A", "label": "Award" },
            { "code": "S", "label": "Sale" },
            { "code": "F", "label": "Tax" },
            { "code": "M", "label": "Exercise" },
            { "code": "G", "label": "Gift" },
            { "code": "P", "label": "Purchase" },
            { "code": "I", "label": "Discretionary" },
            { "code": "C", "label": "Conversion" }
          ]
        },
        "key": "code",
        "fields": ["label"]
      },
      "default": "Unknown"
    }
  ],
  "mark": {
    "type": "circle",
    "opacity": 0.7
  },
  "encoding": {
    "x": {
      "field": "TransactionDate",
      "type": "temporal",
      "title": "Transaction Date"
    },
    "y": {
      "field": "Insider",
      "type": "nominal",
      "title": "Insider",
      "sort": "-x"
    },
    "color": {
      "field": "label",
      "type": "nominal",
      "title": "Transaction Type"
    },
    "size": {
      "field": "SharesTransacted",
      "type": "quantitative",
      "title": "Shares Transacted"
    },
    "tooltip": [
      { "field": "Insider", "type": "nominal", "title": "Insider" },
      { "field": "TransactionDate", "type": "temporal", "title": "Date" },
      { "field": "TransactionCode", "type": "nominal", "title": "Transaction Code" },
      { "field": "SharesTransacted", "type": "quantitative", "title": "Shares Transacted" },
      { "field": "TotalValueDisplay", "type": "nominal", "title": "Total Value ($)" }
    ]
  },
  "config": {
    "axis": {
      "labelFontSize": 10,
      "titleFontSize": 12
    },
    "legend": {
      "labelFontSize": 10,
      "titleFontSize": 12
    },
    "view": {
      "stroke": "transparent"
    }
  }
}
"@ | Out-Display -MimeType "application/vnd.vegalite.v5+json"